In [40]:
% cd "C:\Users\Gerome\Desktop\DMC 2018\Predictions"

import pandas as pd
import numpy as np
from pytrends.request import TrendReq
from functools import reduce
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from sklearn import preprocessing

from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from datetime import datetime

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import plotly.offline as py
import plotly.graph_objs as go
import numpy as np
import seaborn as sns
py.init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings('ignore')

C:\Users\Gerome\Desktop\DMC 2018\Predictions


In [41]:
train_data = pd.read_csv("train.csv", sep='|')
train_data['key'] = train_data['pid'].astype(str) + train_data['size'].astype(str)
train_data['date'] = pd.to_datetime(train_data['date'])
table = pd.pivot_table(train_data, values='units', index=['key'], aggfunc=np.sum)
table.sort_values('units', ascending=False, inplace=True)
relevant_pids = table.loc[table['units'] >= 100]
relevant_pids.reset_index(drop=False, inplace=True)
relevant_pids_list = list(relevant_pids['key'])

In [42]:
item_data = pd.read_csv("items.csv", sep='|')
item_data['key'] = item_data['pid'].astype(str) + item_data['size'].astype(str)

In [43]:
keyword = 'Nike'
pytrend = TrendReq(hl='de-DE')
pytrend.build_payload(kw_list=[keyword], timeframe='2017-10-01 2018-02-28', geo='DE')
Nike_df = pytrend.interest_over_time()
del Nike_df["isPartial"]
Nike_df.reset_index(drop=False, inplace=True)

keyword = 'adidas'
pytrend = TrendReq(hl='de-DE')
pytrend.build_payload(kw_list=[keyword], timeframe='2017-10-01 2018-02-28', geo='DE')
adidas_df = pytrend.interest_over_time()
del adidas_df["isPartial"]
adidas_df.reset_index(drop=False, inplace=True)

keyword = 'PUMA'
pytrend = TrendReq(hl='de-DE')
pytrend.build_payload(kw_list=[keyword], timeframe='2017-10-01 2018-02-28', geo='DE')
PUMA_df = pytrend.interest_over_time()
del PUMA_df["isPartial"]
PUMA_df.reset_index(drop=False, inplace=True)

keyword = 'Jako'
pytrend = TrendReq(hl='de-DE')
pytrend.build_payload(kw_list=[keyword], timeframe='2017-10-01 2018-02-28', geo='DE')
Jako_df = pytrend.interest_over_time()
del Jako_df["isPartial"]
Jako_df.reset_index(drop=False, inplace=True)

keyword = 'Jordan'
pytrend = TrendReq(hl='de-DE')
pytrend.build_payload(kw_list=[keyword], timeframe='2017-10-01 2018-02-28', geo='DE')
Jordan_df = pytrend.interest_over_time()
del Jordan_df["isPartial"]
Jordan_df.reset_index(drop=False, inplace=True)

keyword = 'Sport2000'
pytrend = TrendReq(hl='de-DE')
pytrend.build_payload(kw_list=[keyword], timeframe='2017-10-01 2018-02-28', geo='DE')
Sport2000_df = pytrend.interest_over_time()
del Sport2000_df["isPartial"]
Sport2000_df.reset_index(drop=False, inplace=True)

dfs = [Nike_df, adidas_df, PUMA_df, Jako_df, Jordan_df, Sport2000_df]
df_final = reduce(lambda left,right: pd.merge(left,right,on='date'), dfs)
df_final.set_index('date', inplace=True)

In [45]:
import time
start_time_all = time.time()

all_list = []

for idx, element in enumerate(relevant_pids_list):
    
    start_time_it = time.time()
    
    from numpy.random import seed
    seed(1)
    from tensorflow import set_random_seed
    set_random_seed(2)
    
    
    
    print ('Modelling item ' + element + ' (item no. ' + str(idx+1) + ')' + ' in progress...')
    
    data_dict = {}
    
    data_dict['key'] = element
    
    data_dict['element_in_pids'] = idx
    
    sub_item = item_data.loc[item_data['key'] == element]
    
    sub_train = train_data.loc[train_data['key'] == element]
    sub_train = sub_train.sort_values('date', ascending=True)
    sub_train.reset_index(drop=True, inplace=True)
    
    sub_brand = sub_item.iloc[0]['brand']
    data_dict['brand'] = sub_brand
    sub_stock = int(sub_item.iloc[0]['stock'])
    data_dict['stock'] = sub_stock
    
    date_index = pd.date_range(start='2017-10-01', end='2018-02-28', freq='D')
    date_index = pd.DataFrame(date_index)
    date_index.columns = ['date']
    
    date_sub_train_merged = pd.merge(date_index, sub_train, on='date', how='left')
    
    date_sub_train_merged.set_index('date', inplace=True)
    del date_sub_train_merged['pid']
    del date_sub_train_merged['size']
    
    date_sub_train_merged = date_sub_train_merged.merge(pd.DataFrame(df_final[sub_brand]),left_index=True, right_index=True)
    
    date_sub_train_merged.reset_index(drop=False, inplace=True)
    
    date_sub_train_merged['weekday_name'] = date_sub_train_merged['date'].dt.weekday_name
    
    date_sub_train_merged['is_weekend'] = np.where((date_sub_train_merged['weekday_name'] == 'Saturday') | 
                                               (date_sub_train_merged['weekday_name'] == 'Sunday'), 
                                               1, 0)
    
    model_df = date_sub_train_merged[['units',sub_brand,'weekday_name','is_weekend']]
    
    average_sales = model_df['units'].mean(skipna=True)
    data_dict['average_sales_per_day'] = average_sales
    
    model_df = model_df.merge(pd.DataFrame(date_sub_train_merged['date']),left_index=True, right_index=True)
    
    train_set = model_df[:123]
    predict_set = model_df[123:]

    train_set.dropna(inplace=True)

    model_df = pd.concat([train_set, predict_set])
    
    scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))

    model_df.loc[:,[sub_brand]] = scaler.fit_transform(np.array(model_df.loc[:,[sub_brand]]).reshape(-1,1))

    scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))

    null_index = model_df['units'].isnull()
    model_df.loc[~null_index, ['units']] = scaler.fit_transform(model_df.loc[~null_index, ['units']])

    encoder = preprocessing.LabelEncoder()
    model_df['weekday_name'] = encoder.fit_transform(model_df['weekday_name'])
    model_df['is_weekend'] = encoder.fit_transform(model_df['is_weekend'])

    hotencoder = preprocessing.OneHotEncoder()
    weekday = np.array(model_df['weekday_name']).reshape(-1,1)
    weekend = np.array(model_df['is_weekend']).reshape(-1,1)
    hotencoder.fit(weekday)
    onehot_weekday = hotencoder.transform(weekday).toarray()
    hotencoder.fit(weekend)
    onehot_weekend = hotencoder.transform(weekend).toarray()
    weekday_df = pd.DataFrame(onehot_weekday)
    weekend_df = pd.DataFrame(onehot_weekend)
    
    model_df.reset_index(drop=True, inplace=True)
    
    merge1 = pd.merge(model_df, weekday_df, how='left', left_index=True, right_index=True)
    model_df = pd.merge(merge1, weekend_df, how='left', left_index=True, right_index=True)
    
    del model_df['weekday_name']
    del model_df['is_weekend']
    
    model_df.set_index('date', inplace=True)
    
    model_df.columns = ['units', sub_brand, 'weekday_0', 'weekday_1','weekday_2',
                   'weekday_3','weekday_4','weekday_5', 'weekday_6',
                   'is_weekend_0', 'is_weekend_1']
    
    train_set = model_df[:-28]
    predict_set = model_df[-28:]

    values_predict = predict_set.values
    values_predict = values_predict.astype('float32')
    predict_X = values_predict[:, 1:]
    predict_X = predict_X.reshape((predict_X.shape[0], 1, predict_X.shape[1]))

    values_train = train_set.values
    values_train = values_train.astype('float32')
    
    train_X, train_y = values_train[:, 1:], values_train[:, 0]
    test_X, test_y = values_train[:, 1:], values_train[:, 0]

    # reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
    print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
    
    multi_model = Sequential()
    multi_model.add(LSTM(500, input_shape=(train_X.shape[1], train_X.shape[2])))
    multi_model.add(Dense(1))
    multi_model.compile(loss='mse', optimizer='adam')
    multi_history = multi_model.fit(train_X, train_y, epochs=100, batch_size=30, validation_data=(train_X, train_y), verbose=0, shuffle=False)
    
    yhat = multi_model.predict(test_X)
    test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
    inv_yhat = concatenate((yhat, test_X[:, :1]), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,0]
    
    test_y = test_y.reshape((len(test_y), 1))
    inv_y = concatenate((test_y, test_X[:, :1]), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,0]
    
    rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
    
    data_dict['rmse_full_set'] = rmse
    
    y_predict = multi_model.predict(predict_X)
    predict_X = predict_X.reshape((predict_X.shape[0], predict_X.shape[2]))
    inv_y_predict = scaler.inverse_transform(y_predict)
    inv_y_predict = inv_y_predict[:,0]
    
    import datetime as dt

    start_day = "2018-02-01"

    start_day = dt.datetime.strptime(start_day, '%Y-%m-%d').date()

    sell_off_date = start_day + dt.timedelta(days=int(np.argmin(np.cumsum(inv_y_predict) <= sub_stock)))
    
    data_dict['sell_off_date'] = sell_off_date

    print ('Sell off date for product ' + element + ' is ' + str(sell_off_date))
    
    inv_y_predict_df = pd.DataFrame(inv_y_predict)
    inv_y_predict_df.columns = ['prediction']
    inv_y_predict_df.index = predict_set.index
    
    import string
    
    invalidChars = set(string.punctuation.replace("_", ""))
    
    if any(char in invalidChars for char in element):
        
        inv_y_predict_df.to_csv(str(idx) + '_prediction.csv')
          
    else:
        
        inv_y_predict_df.to_csv(element + '_prediction.csv')
    
    all_list.append(data_dict)
    
    output_df = pd.DataFrame(all_list)
    output_df.to_csv('predictions_output.csv')
    
    print ('Item ' + element + ' completed. ' + str((idx+1)/len(relevant_pids_list)*100) + ' % completed.')
    
    print('Iteration took ' + "%s seconds" % (time.time() - start_time_it))

print('Total time was ' + "%s seconds" % (time.time() - start_time_all))

Modelling item 12985L (item no. 1) in progress...
(123, 1, 10) (123,) (123, 1, 10) (123,)
Sell off date for product 12985L is 2018-02-05
Item 12985L completed. 50.0 % completed.
Iteration took 112.87145590782166 seconds
Modelling item 1041643 1/3 (item no. 2) in progress...
(86, 1, 10) (86,) (86, 1, 10) (86,)
Sell off date for product 1041643 1/3 is 2018-02-06
Item 1041643 1/3 completed. 100.0 % completed.
Iteration took 67.60686683654785 seconds
Total time was180.48432302474976 seconds
